## Prepare model

In [ ]:
from transformers import  GPT2Tokenizer, TFGPT2LMHeadModel, pipeline
from transformers import TFGPT2Model, GPT2Config, TrainingArguments, Trainer
from transformers import TextDataset, glue_convert_examples_to_features
from transformers import TFTrainer, TFTrainingArguments
import tensorflow_datasets as tfds
import numpy as np

import tensorflow as tf
from sklearn.model_selection import train_test_split
import os 
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

## Cleaning the lyrics

In [ ]:
lyrics = []

lyrics += (open('lyrics.txt').readlines())

corpus = [line.strip() for line in corpus if line.strip()!=""]
vocab = sorted(set(' '.join(corpus).split(' ')))

# Train Model

In [ ]:
rnn_corpus = ' '.join(corpus)
vocab = sorted(set(rnn_corpus.split(' ')))

In [ ]:
# Create a mapping from unique characters to indices
word2idx = {u:i for i, u in enumerate(vocab)}
# Make a copy of the unique set elements in NumPy array format for later use in the decoding the predictions
idx2word = np.array(vocab)
# Vectorize the text with a for loop
text_as_int = np.array([word2idx[c] for c in rnn_corpus.split(' ')])

In [ ]:
seq_length = 12

word2idx = {u:i for i, u in enumerate(vocab)}

idx2word = np.array(vocab)

text_as_int = [word2idx[c] for c in rnn_corpus.split(' ')]
seq_as_int = []
for i in range(len(text_as_int) - seq_length):
    seq_as_int += text_as_int[i:i+seq_length]
seq_as_int = np.array(seq_as_int)

In [ ]:
len(seq_as_int)

In [ ]:
word_dataset = tf.data.Dataset.from_tensor_slices(seq_as_int) 
sequences = word_dataset.batch(seq_length, drop_remainder=True)
len(sequences)

In [ ]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
BUFFER_SIZE = 20000 # TF shuffles the data only within buffers

BATCH_SIZE = 512 # Batch size

dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

print(dataset)

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embedding_dim = 256
# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
    vocab_size = len(vocab), # no. of unique characters
    embedding_dim=embedding_dim, # 256
    rnn_units=rnn_units, # 1024
    batch_size=BATCH_SIZE)  # 64 for the traning

model.summary()

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './RNN_training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 30
history = model.fit(dataset, 
                    epochs=EPOCHS, 
                    callbacks=[checkpoint_callback])

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights('./RNN_training_checkpoints/ckpt_20')
model.build(tf.TensorShape([1, None]))
model.summary()

In [ ]:
def generate_text(model, num_generate, temperature, start_string):
    input_eval = [word2idx[s.lower()] for s in start_string.split(' ') if s.lower() in word2idx.keys()] # string to numbers (vectorizing)
    input_eval = tf.expand_dims(input_eval, 0) # dimension expansion
    text_generated = [] # Empty string to store our results
    model.reset_states() # Clears the hidden states in the RNN

    for i in range(num_generate): #Run a loop for number of characters to generate
        predictions = model(input_eval) # prediction for single character
        predictions = tf.squeeze(predictions, 0) # remove the batch dimension

        # using a categorical distribution to predict the character returned by the model
        # higher temperature increases the probability of selecting a less likely character
        # lower --> more predictable
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # The predicted character as the next input to the model
        # along with the previous hidden state
        # So the model makes the next prediction based on the previous character
        input_eval = tf.expand_dims([predicted_id], 0) 
        # Also devectorize the number and add to the generated text
        text_generated.append(' ' + idx2word[predicted_id]) 

    return (start_string + ''.join(text_generated))

In [ ]:
generated_text = generate_text(
                    model, 
                    num_generate=50, 
                    temperature=1, 
                    start_string="Well I m getting pretty tired of being treated like competition When the only one that can hold me down is inside my head")
print(generated_text)

In [ ]:
with open("/home/sas/Desktop/GPT2/lyrics.txt", 'w') as f:
    for line in corpus:
        f.write(line+'\n')